# 2025 데이터 크리에이터 캠프

@PHASE: Mission 1

@TEAM: 최후의 인공지능

## Check GPU Availability

In [ ]:
!nvidia-smi

In [ ]:
# Set CUDA Device Number
DEVICE_NUM = 0
ADDITIONAL_GPU = 0

from os import environ
environ["CUDA_VISIBLE_DEVICES"] = ",".join([f"{i+DEVICE_NUM}" for i in range(0, ADDITIONAL_GPU+1)])
environ["CUDA_VISIBLE_DEVICES"]

## Imports

In [ ]:
from os import path

from creator_camp.datasets import KompsatDataset, SentinelDataset, DatasetHolder

import torch
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from accelerate import Accelerator, notebook_launcher

from supervision.metrics.mean_average_precision import MeanAveragePrecision
from supervision.detection.core import Detections

import wandb
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

In [ ]:
if torch.cuda.is_available():
    if ADDITIONAL_GPU:
        device = torch.device("cuda")
    else:
        device = torch.device(f"cuda")  # torch.device(f"cuda:{DEVICE_NUM}")
else:
    device = torch.device("cpu")
    DEVICE_NUM = -1

print(f"INFO: Using device - {device}" + (f":{DEVICE_NUM}" if ADDITIONAL_GPU else ""))

In [ ]:
PROJECT_NAME = "Mission_1"
RUN_NAME = "RT-DETR"

# WandB Initialization
wandb.init(project=PROJECT_NAME, name=RUN_NAME)

## Define Dataset

In [ ]:
DATA_ROOT = path.join(".", "data")

kompstats = DatasetHolder(train=KompsatDataset(root=DATA_ROOT, train=True), valid=KompsatDataset(root=DATA_ROOT, train=False))
sentinels = DatasetHolder(train=SentinelDataset(root=DATA_ROOT, train=True), valid=SentinelDataset(root=DATA_ROOT, train=False))

In [ ]:
kompstats.train[0]

In [ ]:
sentinels.train[0]

In [ ]:
rgb_image, mask_image = sentinels.train[0]
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].imshow(rgb_image)
axes[0].set_title('Image')
axes[0].axis('off')

axes[1].imshow(mask_image, cmap='gray')
axes[1].set_title("Mask")
axes[1].axis('off')

plt.tight_layout()
plt.show()

## DataLoader

In [ ]:
# Set Batch Size
BATCH_SIZE = 2, 8, 8, 8  # 4070 Ti
#BATCH_SIZE = 32, 64, 64, 32  # A6000

# Dataset Configs
NUM_CLASSES = 1

print(f"INFO: Set batch size - Train: {BATCH_SIZE[0]}, Valid: {BATCH_SIZE[1]}, Test: {BATCH_SIZE[2]}")

In [ ]:
def collate_fn(batch, preprocessor=None):
    images = [item['image'] for item in batch]
    target = [item['target'] for item in batch]
    return preprocessor(images=images, annotations=target, return_tensors="pt")

## Load Model

In [ ]:
from transformers import AutoBackbone, RTDetrForObjectDetection, RTDetrImageProcessorFast, RTDetrConfig
from transformers.image_utils import AnnotationFormat

In [ ]:
reference_model_id = "PekingU/rtdetr_r101vd"

# Load the reference model configuration
reference_config = RTDetrConfig.from_pretrained(reference_model_id, torch_dtype=torch.float32, return_dict=True)
reference_config.num_labels = NUM_CLASSES

# Set the image size and preprocessor size
reference_config.image_size = 640

# Load the reference model image processor
reference_preprocessor = RTDetrImageProcessorFast.from_pretrained(reference_model_id)
reference_preprocessor.format = AnnotationFormat.COCO_DETECTION  # COCO Format / Detection BBOX Format
reference_preprocessor.size = {"height": 640, "width": 640}
reference_preprocessor.do_resize = True
# ---
test_img_size = 896
test_preprocessor = RTDetrImageProcessorFast.from_pretrained(reference_model_id)
test_preprocessor.format = AnnotationFormat.COCO_DETECTION  # COCO Format / Detection BBOX Format
test_preprocessor.size = {"height": test_img_size, "width": test_img_size}
test_preprocessor.do_resize = True

In [ ]:
model = RTDetrForObjectDetection.from_pretrained(reference_model_id, config=reference_config, torch_dtype=torch.float32, ignore_mismatched_sizes=True)
model.to(device)

In [ ]:
from transformers.trainer_utils import EvalPrediction
from torchvision.ops import box_convert
from dataclasses import dataclass


@dataclass
class ModelOutput:
    logits: torch.Tensor
    pred_boxes: torch.Tensor


def de_normalize_boxes(boxes, height, width):
    # 1. cxcywh → xyxy
    boxes_xyxy_norm = box_convert(boxes, 'cxcywh', 'xyxy')

    # 2. de-normalize (convert to actual pixel coordinates)
    boxes_xyxy_norm[:, [0, 2]] *= width
    boxes_xyxy_norm[:, [1, 3]] *= height
    return boxes_xyxy_norm


def map_compute_metrics(preprocessor=reference_preprocessor, threshold=0.0):
    map_metric = MeanAveragePrecision()
    post_process = preprocessor.post_process_object_detection

    def calc(eval_pred: EvalPrediction, compute_result=False):
        nonlocal map_metric

        if compute_result:
            m_ap = map_metric.compute()
            map_metric.reset()

            return {
                "mAP@0.50:0.95": m_ap.map50_95,
                "mAP@0.50": m_ap.map50,
                "mAP@0.75": m_ap.map75
            }
        else:
            preds = ModelOutput(*eval_pred.predictions[1:3])
            labels = eval_pred.label_ids
            sizes = [label['orig_size'].cpu().tolist() for label in labels]

            results = post_process(preds, target_sizes=sizes, threshold=threshold)
            predictions = [Detections.from_transformers(result) for result in results]
            targets = [Detections(
                xyxy=de_normalize_boxes(label['boxes'], *label['orig_size']).cpu().numpy(),
                class_id=label['class_labels'].cpu().numpy(),
            ) for label in labels]

            map_metric.update(predictions=predictions, targets=targets)
            return {}
    return calc, map_metric

## Train

In [ ]:
# Set Epoch Count & Learning Rate
EPOCHS = 20
REAL_BATCH = BATCH_SIZE[-1]
LEARNING_RATE = 1e-4

training_args = TrainingArguments(
    backbone_lr=LEARNING_RATE/10,  # Set backbone learning rate to 1/10th of the main learning rate
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    weight_decay=0.1,
    max_grad_norm=0.5,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE[0],
    per_device_eval_batch_size=BATCH_SIZE[1],
    gradient_accumulation_steps=REAL_BATCH//BATCH_SIZE[0],
    eval_accumulation_steps=BATCH_SIZE[1],
    batch_eval_metrics=True,
    remove_unused_columns=False,
    optim="adamw_torch",
    eval_strategy="steps",
    save_strategy="steps",
    logging_strategy="steps",
    eval_steps=100,
    save_steps=100,
    logging_steps=100,
    save_total_limit=100,
    load_best_model_at_end=True,
    metric_for_best_model="mAP@0.50",
    greater_is_better=True,
    #metric_for_best_model="eval_loss",
    #greater_is_better=False,
    report_to="wandb",
    output_dir="./results/"+RUN_NAME,
    logging_dir="./logs/"+RUN_NAME,
    run_name=RUN_NAME,
    bf16=True,
)

testing_args = TrainingArguments(
    per_device_eval_batch_size=BATCH_SIZE[2],
    batch_eval_metrics=True,
    remove_unused_columns=False,
)

In [ ]:
from functools import partial

compute_metrics, compute_results = map_compute_metrics(preprocessor=reference_preprocessor)
test_compute_metrics, test_compute_results = map_compute_metrics(preprocessor=test_preprocessor)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=kompstats.train,
    eval_dataset=kompstats.valid,
    data_collator=partial(collate_fn, preprocessor=reference_preprocessor),
    compute_metrics=compute_metrics,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=30)]
)

tester = Trainer(
    model=model,
    args=testing_args,
    eval_dataset=kompstats.valid,
    data_collator=partial(collate_fn, preprocessor=test_preprocessor),
    compute_metrics=test_compute_metrics
)

In [ ]:
def start_train():
    accelerator = Accelerator()
    while True:
        try:
            try:
                print("INFO: Trying to resume from previous checkpoint")
                compute_results.reset()
                trainer.train(resume_from_checkpoint=True)
            except Exception as e:
                if "No valid checkpoint found" in str(e):
                    print(f"ERROR: Failed to resume from checkpoint - {e}")
                    print("INFO: Starting training from scratch")
                    compute_results.reset()
                    trainer.train(resume_from_checkpoint=False)
        except Exception as e:
            if "CUDA" in str(e):
                print(f"ERROR: CUDA Error - {e}")
                trainer.train()
            else:
                raise e

In [ ]:
if ADDITIONAL_GPU:
    notebook_launcher(start_train, args=(), num_processes=ADDITIONAL_GPU)
else:
    start_train()

## Evaluate

In [ ]:
trainer.evaluate()

In [ ]:
tester.evaluate()

In [ ]:
checkpoint = 31100

In [ ]:
try:
    model = RTDetrForObjectDetection.from_pretrained(f"{training_args.output_dir}/checkpoint-{checkpoint}/", torch_dtype=torch.float32, return_dict=True, local_files_only=True)
    model.to(device)
except Exception:
    pass